In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv('Dataset/Truth_Seeker_Model_Dataset.csv')
df

In [ ]:
df.drop(columns=['target'], inplace=True)

In [ ]:
print(df['3_label_majority_answer'].unique())
print(df['5_label_majority_answer'].unique())

In [ ]:
true_source_df = df[df['BinaryNumTarget'] == 1].copy()
false_source_df = df[df['BinaryNumTarget'] == 0].copy()

In [ ]:
label_mapping_true_binary = {
    'Agree': 1,
    'Disagree': 0

}

label_mapping_false_binary = {
    'Agree': 0,
    'Disagree': 1,
}

true_source_df['target_binary'] = true_source_df['3_label_majority_answer'].replace(label_mapping_true_binary)
false_source_df['target_binary'] = false_source_df['3_label_majority_answer'].replace(label_mapping_false_binary)

In [ ]:
label_mapping_true_5 = {
    'Agree': 'true',
    'Mostly Agree': 'mostly_true',
    'NO MAJORITY': 'no_majority',
    'Mostly Disagree': 'mostly_false',
    'Disagree': 'false'
}

label_mapping_false_5 = {
    'Agree': 'false',
    'Mostly Agree': 'mostly_false',
    'NO MAJORITY': 'no_majority',
    'Mostly Disagree': 'mostly_true',
    'Disagree': 'true'
}

true_source_df['target_5'] = true_source_df['5_label_majority_answer'].replace(label_mapping_true_5)
false_source_df['target_5'] = false_source_df['5_label_majority_answer'].replace(label_mapping_false_5)

In [ ]:
df = pd.concat([true_source_df, false_source_df], ignore_index=True)
df

In [ ]:
label_mapping_3 = {
    'true': 'true',
    'mostly_true': 'true',
    'no_majority': 'no_majority',
    'mostly_false': 'false',
    'false': 'false'
}

df['target_3'] = df['target_5'].replace(label_mapping_3)
df

In [ ]:
df = df[['tweet', 'manual_keywords', 'statement', 'target_binary', 'target_3', 'target_5']].copy()
df = df.rename(columns={'manual_keywords': 'keywords'}).copy()
df

In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def preprocess_text(text, number_indicator="NUM", url_indicator="URL", percent_indicator="PERCENT", money_indicator="MONEY"):
    
    punctuation = r"[\]^_`{|}&<>*#-:"

    # Function to expand contractions
    def replace_contractions(text):
        contractions_pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contraction_dict.keys()) + r')\b', re.IGNORECASE)

        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_dict[match.lower() if match.lower() in contraction_dict else match]
            return first_char + expanded_contraction[1:] if first_char.isupper() else expanded_contraction

        return contractions_pattern.sub(expand_match, text)

    # Expand contractions
    text = replace_contractions(text)

    # Remove punctuation
    for p in punctuation:
        text = text.replace(p, "")

    # Replace URLs with URL
    url_pattern = r'https?//(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_pattern, url_indicator, text)

    # Label MONEY
    text = re.sub(r'\$\b(?!0+(?:\.0+)?$)\d+(\.\d{1,2})?\b', lambda m: money_indicator, text)

    # Replace numbers, except years, with NUM 
    text = re.sub(r'\b\d{1,3}\b', number_indicator, text)
    text = re.sub(r'\b(?!1|2)\d{4,}\b', number_indicator, text)
    
    # Label PERCENT
    text = text.replace(number_indicator + "%", percent_indicator)
    
    # Replace newline with a space and remove mentions
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'@\w+', '', text)

    return text

In [ ]:
example_text = "I'm sure it'll be great in 2024, but it costs $20. Visit https://example.com or call 123 for details. There's a 100% chance!"
processed_text = preprocess_text(example_text)
print(processed_text)

In [ ]:
%%time
df['tweet'] = df['tweet'].apply(lambda x: pd.Series(preprocess_text(x)))
df

In [ ]:
df.to_json('Processed/processed_all.json', orient='records')

In [ ]:
# Split into individual Keywords
df['keywords'] = df['keywords'].str.split(', ')

# Explode the Dataframe
df_exploded = df.explode('keywords')
df_exploded

In [ ]:
# Extract COVID specific tweets
covid_df = df_exploded[df_exploded['keywords'].str.contains('COVID|COVID-19|covid|covid-19|corona|pandemic|virus', case=False, na=False)]
covid_df

In [ ]:
covid_df['tweet'] = covid_df['tweet'].drop_duplicates()
covid_df = covid_df.dropna(subset=['tweet'])

In [ ]:
covid_df["target_binary"].value_counts()

In [ ]:
# Drop val Tweets from DF
df = df.drop(covid_df.index)
df

In [ ]:
df

In [ ]:
def balance_dataset_by_source_statement(df):
    # Initialize an empty list to store the balanced dataframes
    balanced_dfs = []

    # Group by the 'keywords_str' column
    grouped = df.groupby('statement')

    # Iterate over each group
    for name, group in grouped:
        false = group[group['target_binary'] == 0]  # Filtering false samples
        true = group[group['target_binary'] == 1]   # Filtering true samples
        
        # Determine the minimum number of samples
        minimum = min(len(false), len(true))
        
        # Concatenate the balanced samples
        result = pd.concat([false.head(minimum), true.head(minimum)])
    
        balanced_dfs.append(result)

    # Concatenate all balanced dataframes in the list
    balanced_df = pd.concat(balanced_dfs)
    
    return balanced_df

balanced_df = balance_dataset_by_source_statement(df)

In [ ]:
balanced_df = balanced_df.sort_index()
balanced_df

In [ ]:
# Binary
df[['tweet', 'target_binary']].to_json('Processed/processed_binary.json', orient='records')
covid_df[['tweet', 'target_binary']].to_json('Processed/processed_binary_val.json', orient='records')

# Multiclass
df[['tweet', 'target_3']].to_json('Processed/processed_multiclass_3.json', orient='records')
covid_df[['tweet', 'target_3']].to_json('Processed/processed_multiclass_3_val.json', orient='records')

df[['tweet', 'target_5']].to_json('Processed/processed_multiclass_5.json', orient='records')
covid_df[['tweet', 'target_5']].to_json('Processed/processed_multiclass_5_val.json', orient='records')